# Intertidal Bathymetry
Tests intertidal bathymetry function of `bathymetry.py`. 

In [1]:
from pathlib import Path
import sys
import time

import ee
import geemap
from geojson import Polygon

sys.path.append(str(Path.cwd().parent.parent))

from eepackages.applications import bathymetry

In [2]:
Map = geemap.Map(center=(52.97, 4.74), zoom=11)
bounds = ee.Geometry(Polygon([[
    (4.574518432617185, 52.91662291147939),
    (4.90548156738281, 52.91662291147939),
    (4.90548156738281, 53.02331125248889),
    (4.574518432617185, 53.02331125248889),
    (4.574518432617185, 52.91662291147939)
]]))

In [3]:
sdb = bathymetry.Bathymetry()

scale = Map.getScale()

# Parameters 
start_date = '2018-01-01'
stop_date = '2020-01-01'

image = sdb.compute_intertidal_depth(
    bounds=bounds,
    start=start_date,
    stop=stop_date,
    scale=scale,
    missions=['S2', 'L8'],
    # filter: ee.Filter.dayOfYear(7*30, 9*30), # summer-only
    filter_masked=False, 
    # filterMasked: true, 
    # filterMaskedFraction: 0.5,
    skip_scene_boundary_fix=False,
    skip_neighborhood_search=False,
    neighborhood_search_parameters={"erosion": 0, "dilation": 0, "weight": 50},
    bounds_buffer=0,
    water_index_min=-0.05,
    water_index_max=0.15,

    # lowerCdfBoundary: 45,
    # upperCdfBoundary: 50
)  # .reproject(ee.Projection("EPSG:3857").atScale(90))

min = 0.04
max = 0.2
Map.addLayer(sdb.composite, { "min": min, "max": max }, 'red-green-blue')
Map

Map(center=[52.97, 4.74], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children…

In [ ]:
Map.addLayer(image, {"min": sdb.waterIndexMin, "max": sdb.waterIndexMax}, 'water indices', False)